In [ ]:
# !pip install openpyxl

In [10]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.

In [ ]:
import numpy as np
import pandas as pd


file_path = '/content/FInal_data_20_columns.xlsx'
df = pd.read_excel(file_path)

df = df.astype(str)
df = df.fillna("")
df = df[:1000]
df.head(2)

,ChiefComplaint,PresentProblemLocation,PreProbDuration,PreProbOnset,PreProbSeverity,AttackType,SmokingHabitType,PhysicalActivity,ALCHOLIC,DietryHabit,...,TreatmentType,SurgeryType,SurgeryDurationH,SurgeryDurationM,AnesthicTypeName,AnesRecoverySmoothType,RecorveryType,Followupdate,DoctorRemark,Paragraph
0,Ho intermittent pain abdomen and flatulence X ...,Abdomen problem,Complains from 1 month ago,Gradually it happen,nan,Intermittent attack,Patient does not have smoking habit,Light Worker,Does not drink alcohol,nan,...,Treatment type OPERATIVE,Surgery type CHOLECYSTECTOMY,nan,40 minutes,Anesthic type General,Anes smooth recovery,NORMAL recovery,Follow-up date 20 December 2014,nan,Ho intermittent pain abdomen and flatulence X ...
1,During routine health checkup gallstone detect...,Abdomen problem,Complains from 2 months ago,nan,Mild problem,No attack,Patient does not have smoking habit,Moderate Worker,Drinks alcohol,Non Vegetarian,...,Treatment type OPERATIVE,Surgery type CHOLECYSTECTOMY,nan,30 minutes,Anesthic type General,Anes smooth recovery,NORMAL recovery,Follow-up date 20 December 2014,nan,During routine health checkup gallstone detect...


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train and validation
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Prepare data for training
train_sentences = []
train_labels = []

for index, row in train_df.iterrows():
    for column in row.index:
        if column != 'Paragraph':
            train_sentences.append(row[column])
            train_labels.append(column)

In [ ]:
len(train_sentences)

16800

In [ ]:
len(train_labels)

16800

In [ ]:
val_sentences = []
val_labels = []

for index, row in val_df.iterrows():
    for column in row.index:
        if column != 'Paragraph':
            val_sentences.append(row[column])
            val_labels.append(column)

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

# Load pre-trained GPT-2 tokenizer and model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# from datasets import Dataset

# def create_dataset(df):
#     texts = df['Paragraph'].tolist()
#     labels = [' '.join([f"{label}: {sentence}" for label, sentence in zip(df.columns[:-1], row[:-1])]) for row in df.itertuples(index=False)]
    
#     # Create a list of dictionaries
#     data = [{'text': f"{text} => {label}"} for text, label in zip(texts, labels)]
#     return Dataset.from_list(data)

# # Create Hugging Face datasets from DataFrames
# train_dataset = create_dataset(train_df)
# val_dataset = create_dataset(val_df)

# # Tokenize datasets
# def tokenize_function(examples):
#     return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

# tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
# tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# # Initialize data collator
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer, mlm=False
# )

In [ ]:
# Tokenize data
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

# # Create dataset from pandas DataFrame
# def dataset_from_dataframe(df, tokenizer):
#     texts = df['Paragraph'].tolist()
#     labels = [' '.join([f"{label}: {sentence}" for label, sentence in zip(df.columns[:-1], row[:-1])]) for row in df.itertuples(index=False)]
#     data = {'text': [f"{text} => {label}" for text, label in zip(texts, labels)]}
#     return TextDataset(
#         tokenizer=tokenizer,
#         file_path=None,
#         block_size=128,
#         overwrite_cache=True,
#         dataset=data,
#         cache_dir="./cached_dataset"
#     )
# train_dataset = dataset_from_dataframe(train_df, tokenizer)
# val_dataset = dataset_from_dataframe(val_df, tokenizer)



# Function to create a temporary text file for TextDataset
def dataset_from_dataframe(df, tokenizer, tmp_file_path):
    texts = df['Paragraph'].tolist()
    labels = [' '.join([f"{label}: {sentence}" for label, sentence in zip(df.columns[:-1], row[:-1])]) for row in df.itertuples(index=False)]

    with open(tmp_file_path, 'w', encoding='utf-8') as f:
        for text, label in zip(texts, labels):
            f.write(f"{text} => {label}\n")

    return TextDataset(
        tokenizer=tokenizer,
        file_path=tmp_file_path,
        block_size=128,
        overwrite_cache=True
    )

In [ ]:
# File paths for temporary storage
train_file_path = './train_data.txt'
val_file_path = './val_data.txt'

train_dataset = dataset_from_dataframe(train_df, tokenizer, train_file_path)
val_dataset = dataset_from_dataframe(val_df, tokenizer, val_file_path)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
# # Clean up temporary files
# os.remove(train_file_path)
# os.remove(val_file_path)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_steps=400,
    save_steps=800,
    warmup_steps=500,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    # weight_decay=0.01,  ## weight_decay =>  L2 regularization
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tune the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained('./gpt2-finetuned-model')
tokenizer.save_pretrained('./gpt2-finetuned-tokenizer')

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.803200,0.686477
2,0.593500,0.612256


Epoch,Training Loss,Validation Loss
1,0.803200,0.686477
2,0.593500,0.612256
3,0.505500,0.600393


('./gpt2-finetuned-tokenizer/tokenizer_config.json',
 './gpt2-finetuned-tokenizer/special_tokens_map.json',
 './gpt2-finetuned-tokenizer/vocab.json',
 './gpt2-finetuned-tokenizer/merges.txt',
 './gpt2-finetuned-tokenizer/added_tokens.json')

In [ ]:
model = GPT2LMHeadModel.from_pretrained('./gpt2-finetuned-model')
tokenizer = GPT2Tokenizer.from_pretrained('./gpt2-finetuned-tokenizer')


def generate_labels(paragraph):
    input_text = f"{paragraph} =>"
    inputs = tokenizer.encode(input_text, return_tensors='pt')
    outputs = model.generate(inputs, max_length=512, num_return_sequences=1, num_beams=5, early_stopping=True)
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_text.split('=>')[-1].strip()

# Example paragraph
paragraph = "ho intermittent pain abdomen and flatulence x 1 month . abdomen problem . complains from 1 month ago . gradually it happen . intermittent attack . patient does not have smoking habit . light worker . does not drink alcohol . past medical history non diabetic. non hypertensive. she underwent lscs in 2005 . cholelithiasis . treatment type operative . surgery type cholecystectomy . procedure follow cholecystectomy . 40 minutes . anesthic type general . anes smooth recovery . gb found to be well distended. gb was retracted and calot s area was disected. cystic duct artery clipped cut. gb disected out from the liver bed.it is extracted through the epigastric port. ports were closed. gb tissue sent for biopsy . normal recovery . follow-up date 20 december 2014 ."

# Generate labels
result = generate_labels(paragraph)
print(result)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


ChiefComplaint: Ho intermittent pain abdomen and flatulence x 1 month PresentProblemLocation: Abdomen problem PreProbDuration: Complains from 1 month ago PreProbOnset: Gradually it happen PreProbSeverity: Moderate problem AttackType: No attack SmokingHabitType: Patient does not have smoking habit PhysicalActivity: Light Worker ALCHOLIC: Does not drink alcohol DietryHabit: Non Vegetarian MealFrequency: Prefer meals 2 3 times a day VegCookingMedia: Prefer oil mustrad oil TreatmentType: Treatment type OPERATIVE SurgeryType: Surgery type CHOLECYSTECTOMY SurgeryDurationH: nan SurgeryDurationM: nan AnesthicTypeName: Anesthic type General AnesRecoverySmoothType: Anes smooth recovery RecorveryType: NORMAL recovery Followupdate: Follow-up date 20 December 2014 DoctorRemark: CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS
Ho intermittent pain abdomen and flatulence x 1 month PresentProblemLocation: Abdomen problem PreProbOnset: Gradually it happen PreProbSeverity: Moderate problem AttackType: No attack

In [ ]:
# paragraph = "ho intermittent pain abdomen and flatulence x 1 month . abdomen problem . complains from 1 month ago . gradually it happen . intermittent attack . patient does not have smoking habit . light worker . does not drink alcohol . past medical history non diabetic. non hypertensive. she underwent lscs in 2005 . cholelithiasis . treatment type operative . surgery type cholecystectomy . procedure follow cholecystectomy . 40 minutes . anesthic type general . anes smooth recovery . gb found to be well distended. gb was retracted and calot s area was disected. cystic duct artery clipped cut. gb disected out from the liver bed.it is extracted through the epigastric port. ports were closed. gb tissue sent for biopsy . normal recovery . follow-up date 20 december 2014 ."
# paragraph = "Pain abdomen . Abdomen problem . Complains from 1 month ago . Gradually it happen . Mild problem . No attack . Chronic appendicitis . Treatment type OPERATIVE . Surgery type APPENDICECTOMY . Procedure follow LAPAROSOPIC CHOLECYSTECTOMY . Anesthic type General . Anes smooth recovery . Follow-up date 18 September 2015 . CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS . "
# paragraph = "intermittent acute pain in epigastric region radiating to back since 6 months. It was associated with flatulence  nausea. LSCS in 2oo8. irregular bowel . Abdomen problem . Complains from 6 months ago . Gradually it happen . Mild problem . Intermittent attack . Patient does not have smoking habit . Does not drink alcohol . Both VegNon Veg . Prefer oil mustrad oil . Past medical history LSCS in 2008 . Acute cal. cholecystitis with lower abdominal omental adhesions . Treatment type OPERATIVE . Surgery type CHOLECYSTECTOMY . Procedure follow LAPAROSOPIC CHOLECYSTECTOMY . Anesthic type General . Anes smooth recovery . Recommende drug INJ. PYROLATE 1 AMP .  INJ. PROPOFOL . GB inflammed, thick walled Calot s area inflammed Cystic duct contains a calculus Lower abdominal omento bowel adhesions Multiple calculi CBD normal . NORMAL recovery . Medicines suggested Normal 0 false false false EN IN X NONE X NONE MicrosoftInternetExplorer4 lt;   [if gte mso 9]gt; lt;wLsdException Locked= . Follow-up date 29 March 2013 . CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS . "
# paragraph = "Ho right hypochondrium discomfort since 5 6 months. Umbilical hernia detected in 20 December 2014. She is non diabetic and non hypertensive. She underwent LSCS 6 years ago. USG abdomen on 20 December 2014 revealed Umbilical hernia, benign hepatic cysts with pressure effect in RHD  fibroid. MDCT on 31 December 2014 showed hepatomegaly with asymmetrical fatty infiltration and multiple non enhancing cystic lesions in the right and caudate lobes showing regular peripheral rim without enhancing mural solid component. Suggest possibilities like biliary cyst adenomas  hepatic cysts. The largest lesion is compressing the right hepatic duct with prominence of corresponding biliary radicles. There are few prominent paraaortic and interaortocaval nodes. Gastric rugae are prominent. Visualized bowel loops show no significant wall thickening or dilatation. There is umbilical hernia. Patient again admitted on 03 January 2017 with history of right upper abdominal discomfort. No ho fever. She had past ho umbilical hernia repair and drainage of right simple liver cyst done in 2015. She is known hypertensive and on treatment. USG abdomen on 30 December 2016 revealed benign hepatic cyst  hydatid . Abdomen problem . Complains from 6 months ago . Gradually it happen . Mild problem . No attack . Patient does not have smoking habit . Light Worker . Does not drink alcohol . Both VegNon Veg . Prefer meals 2 3 times a day . Prefer oil mustrad oil . Past medical history LSCS 6 years ago . No medicine refered by other . Normal 0 MicrosoftInternetExplorer4  Style Definitions  table.MsoNormalTable mso style name Table Normal ; mso tstyle rowband size0; mso tstyle colband size0; mso style noshowyes; mso style parent""; mso padding alt0in 5.4pt 0in 5.4pt; mso para margin0in; mso para margin bottom.0001pt; mso paginationwidow orphan; font size10.0pt; font family Times New Roman ; 17 January 2015  Simple hepatic cyst segment Iv V with umbilical hernia omentocele 04 January 2017  Hepatic cyst rt liver segment VVI Hypertension . Treatment type OPERATIVE . Surgery type MULTIPLE . Procedure follow LAP DRAINAGE OF CYST OF CYST  LAP. ADHESIOLYSIS, RELEASE OF OMENTOCELE  MESH REPAIR OF HERNIA . Anesthic type General . Anes smooth recovery . Recommende drug INJ. PYROLATE 1 AMP .  INJ. PROPOFOL .  17 January 2015Lap. drainage of cyst lap. adhesiolysis, release of omentocele mesh repair of hernia  Lap. adhesiolysis of porta and hepatic bed. Caudate lobe Segment V VI dissected and lifting up. Deep seated cyst aspirated out. Hernia contents released using harmonic scalpel. 12x12 cm parietex mesh applied fixed with tacjer suture. 04 January 2017  Difficult lap. exploration hydatid cyst with marsupialization cholecystectomy   Cyst is situated in the postero inferior surface lateral to porta overlying upper pole of rt kidney Segment VVI. Cyst could not be exposed as GB was hindering the exploration. Lap. Cholecystectomy was done. Identification cyst was difficult as there was overlying liver tissue. Needle aspiration of cyst revealed golden bile stained fluid. Cyst was opened sucked out the scolices germinal layer. Cavity was treated with 3 NS 10 povidene iodine. Marsupialization of cyst was done. Active bleeding from cyst wall was controlled with harmonic scalpel. Margin was sutured. Operative field was irrigated with NS. The bile communication with the cyst could not be identified. Drainage applied within Sac ports were closed. Post operative biliary drainage was collected measured through drainage tube. Tissues sent for Biopsy . NORMAL recovery . Medicines suggested INJ. AMIKACIN 750 INJ. KETONOV 2AMP INJ. PANTOCID 40 . Follow-up date 28 January 2015 . CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS AVOID STRAIN . NO . "

In [ ]:
def generate_labels(sent):
    paragraph = sent
    input_text = f"{paragraph} =>"
    inputs = tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True)

    # Generate predictions
    outputs = model.generate(
        inputs,
        max_length=512,
        num_return_sequences=1,
        num_beams=5,
        early_stopping=True
    )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    labels = output_text.split('=>')[-1].strip()

    return {"labels": labels}


generate_labels("intermittent acute pain in epigastric region radiating to back since 6 months. It was associated with flatulence  nausea. LSCS in 2oo8. irregular bowel . Abdomen problem . Complains from 6 months ago . Gradually it happen . Mild problem . Intermittent attack . Patient does not have smoking habit . Does not drink alcohol . Both VegNon Veg . Prefer oil mustrad oil . Past medical history LSCS in 2008 . Acute cal. cholecystitis with lower abdominal omental adhesions . Treatment type OPERATIVE . Surgery type CHOLECYSTECTOMY . Procedure follow LAPAROSOPIC CHOLECYSTECTOMY . Anesthic type General . Anes smooth recovery . Recommende drug INJ. PYROLATE 1 AMP .  INJ. PROPOFOL . GB inflammed, thick walled Calot s area inflammed Cystic duct contains a calculus Lower abdominal omento bowel adhesions Multiple calculi CBD normal . NORMAL recovery . Medicines suggested Normal 0 false false false EN IN X NONE X NONE MicrosoftInternetExplorer4 lt;   [if gte mso 9]gt; lt;wLsdException Locked= . Follow-up date 29 March 2013 . CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS . ")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'labels': 'ChiefComplaint: Ho intermittent pain in epigastric region radiating to back since 6 months PresentProblemLocation: Abdomen problem PreProbDuration: Complains from 6 months ago PreProbOnset: Gradually it happen PreProbSeverity: Moderate problem AttackType: No attack SmokingHabitType: Patient does not have smoking habit PhysicalActivity: Light Worker ALCHOLIC: Does not drink alcohol DietryHabit: Non Vegetarian MealFrequency: Prefer meals 2 3 times a day VegCookingMedia: Prefer oil mustrad oil TreatmentType: Treatment type CHOLECYSTECTOMY SurgeryType: Surgery type CHOLECYSTECTOMY SurgeryDurationH: nan SurgeryDurationM: nan AnesthicTypeName: Anesthic type General AnesRecoverySmoothType: Anes smooth recovery RecorveryType: NORMAL recovery Followupdate: Follow-up date 29 March 2013 DoctorRemark: CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS\nHo intermittent pain in ep'}

In [ ]:
def parse_output(output):
    lines = output.split(' ')
    labeled_sentences = {}
    current_label = None
    for line in lines:
        if ':' in line:
            current_label, sentence = line.split(':', 1)
            labeled_sentences[current_label.strip()] = sentence.strip()
        else:
            if current_label:
                labeled_sentences[current_label] += ' ' + line.strip()
    return labeled_sentences

# Example output parsing
parsed_result = parse_output(result)
print(parsed_result)


{'ChiefComplaint': ' Ho intermittent pain abdomen and flatulence x 1 month', 'PresentProblemLocation': ' Abdomen problem', 'PreProbDuration': ' Complains from 1 month ago', 'PreProbOnset': ' Gradually it happen', 'PreProbSeverity': ' Moderate problem', 'AttackType': ' No attack', 'SmokingHabitType': ' Patient does not have smoking habit', 'PhysicalActivity': ' Light Worker', 'ALCHOLIC': ' Does not drink alcohol', 'DietryHabit': ' Non Vegetarian', 'MealFrequency': ' Prefer meals 2 3 times a day', 'VegCookingMedia': ' Prefer oil mustrad oil', 'TreatmentType': ' Surgery type CHOLECYSTECTOMY', 'SurgeryType': ' Surgery type CHOLECYSTECTOMY', 'SurgeryDurationH': ' nan', 'SurgeryDurationM': ' nan', 'AnesthicTypeName': ' Anesthic type General', 'AnesRecoverySmoothType': ' Anes smooth recovery', 'RecorveryType': ' NORMAL recovery', 'Followupdate': ' Follow-up date 20 December 2014', 'DoctorRemark': ' CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS\nHo intermittent pain abdomen and flatulence x 1 month

In [ ]:
def parse_output(output):
    lines = output.split(' ')
    labeled_sentences = {}
    current_label = None
    for line in lines:
        if ':' in line:
            current_label, sentence = line.split(':', 1)
            labeled_sentences[current_label.strip()] = sentence.strip()
        else:
            if current_label:
                labeled_sentences[current_label] += ' ' + line.strip()
    return labeled_sentences

# Example output parsing
parsed_result = parse_output('ChiefComplaint: Ho intermittent pain in epigastric region radiating to back since 6 months PresentProblemLocation: Abdomen problem PreProbDuration: Complains from 6 months ago PreProbOnset: Gradually it happen PreProbSeverity: Moderate problem AttackType: No attack SmokingHabitType: Patient does not have smoking habit PhysicalActivity: Light Worker ALCHOLIC: Does not drink alcohol DietryHabit: Non Vegetarian MealFrequency: Prefer meals 2 3 times a day VegCookingMedia: Prefer oil mustrad oil TreatmentType: Treatment type CHOLECYSTECTOMY SurgeryType: Surgery type CHOLECYSTECTOMY SurgeryDurationH: nan SurgeryDurationM: nan AnesthicTypeName: Anesthic type General AnesRecoverySmoothType: Anes smooth recovery RecorveryType: NORMAL recovery Followupdate: Follow-up date 29 March 2013 DoctorRemark: CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS\nHo intermittent pain in ep')

print(parsed_result)

{'ChiefComplaint': ' Ho intermittent pain in epigastric region radiating to back since 6 months', 'PresentProblemLocation': ' Abdomen problem', 'PreProbDuration': ' Complains from 6 months ago', 'PreProbOnset': ' Gradually it happen', 'PreProbSeverity': ' Moderate problem', 'AttackType': ' No attack', 'SmokingHabitType': ' Patient does not have smoking habit', 'PhysicalActivity': ' Light Worker', 'ALCHOLIC': ' Does not drink alcohol', 'DietryHabit': ' Non Vegetarian', 'MealFrequency': ' Prefer meals 2 3 times a day', 'VegCookingMedia': ' Prefer oil mustrad oil', 'TreatmentType': ' Treatment type CHOLECYSTECTOMY', 'SurgeryType': ' Surgery type CHOLECYSTECTOMY', 'SurgeryDurationH': ' nan', 'SurgeryDurationM': ' nan', 'AnesthicTypeName': ' Anesthic type General', 'AnesRecoverySmoothType': ' Anes smooth recovery', 'RecorveryType': ' NORMAL recovery', 'Followupdate': ' Follow-up date 29 March 2013', 'DoctorRemark': ' CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS\nHo intermittent pain in ep'}


In [ ]:
def generate_labels(sent):
    paragraph = sent
    input_text = f"{paragraph} =>"
    inputs = tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True)

    # Generate predictions
    outputs = model.generate(
        inputs,
        max_length=512,
        num_return_sequences=1,
        num_beams=1,
        early_stopping=True
    )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    labels = output_text.split('=>')[-1].strip()

    return labels


result = generate_labels("intermittent acute pain in epigastric region radiating to back since 6 months. It was associated with flatulence  nausea. LSCS in 2oo8. irregular bowel . Abdomen problem . Complains from 6 months ago . Gradually it happen . Mild problem . Intermittent attack . Patient does not have smoking habit . Does not drink alcohol . Both VegNon Veg . Prefer oil mustrad oil . Past medical history LSCS in 2008 . Acute cal. cholecystitis with lower abdominal omental adhesions . Treatment type OPERATIVE . Surgery type CHOLECYSTECTOMY . Procedure follow LAPAROSOPIC CHOLECYSTECTOMY . Anesthic type General . Anes smooth recovery . Recommende drug INJ. PYROLATE 1 AMP .  INJ. PROPOFOL . GB inflammed, thick walled Calot s area inflammed Cystic duct contains a calculus Lower abdominal omento bowel adhesions Multiple calculi CBD normal . NORMAL recovery . Medicines suggested Normal 0 false false false EN IN X NONE X NONE MicrosoftInternetExplorer4 lt;   [if gte mso 9]gt; lt;wLsdException Locked= . Follow-up date 29 March 2013 . CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS . ")

def parse_output(output):
    lines = output.split(' ')
    labeled_sentences = {}
    current_label = None
    for line in lines:
        if ':' in line:
            current_label, sentence = line.split(':', 1)
            labeled_sentences[current_label.strip()] = sentence.strip()
        else:
            if current_label:
                labeled_sentences[current_label] += ' ' + line.strip()
    return labeled_sentences

# Example output parsing
parsed_result = parse_output(result)

print(parsed_result)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'ChiefComplaint': ' Ho intermittent pain in epigastric region radiating to back since 6 months', 'PresentProblemLocation': ' Abdomen problem', 'PreProbDuration': ' Complains from 6 months ago', 'PreProbOnset': ' Gradually it happen', 'PreProbSeverity': ' Moderate problem', 'AttackType': ' No attack', 'SmokingHabitType': ' Patient does not have smoking habit', 'PhysicalActivity': ' Light Worker', 'ALCHOLIC': ' Does not drink alcohol', 'DietryHabit': ' Non Vegetarian', 'MealFrequency': ' Prefer meals 2 3 times a day', 'VegCookingMedia': ' Prefer oil mustrad oil', 'TreatmentType': ' Treatment type CHOLECYSTECTOMY', 'SurgeryType': ' Surgery type CHOLECYSTECTOMY', 'SurgeryDurationH': ' nan', 'SurgeryDurationM': ' nan', 'AnesthicTypeName': ' Anesthic type General', 'AnesRecoverySmoothType': ' Anes smooth recovery', 'RecorveryType': ' NORMAL recovery', 'Followupdate': ' Follow-up date 30 March 2013', 'DoctorRemark': ' CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS\nHo intermittent pain in ep'}


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Example model and tokenizer saving paths
model_save_path = '/content/drive/MyDrive/gpt2-finetuned-model'
tokenizer_save_path = '/content/drive/MyDrive/gpt2-finetuned-tokenizer'

# Make directories if they don't exist
os.makedirs(model_save_path, exist_ok=True)
os.makedirs(tokenizer_save_path, exist_ok=True)

# Save model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Model and tokenizer saved to {model_save_path} and {tokenizer_save_path}")


In [8]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/gpt2-finetuned-model')
tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/gpt2-finetuned-tokenizer')

def generate_labels(sent):
    paragraph = sent
    input_text = f"{paragraph} =>"
    inputs = tokenizer.encode(input_text, return_tensors='pt', max_length=512, truncation=True)

    # Generate predictions
    outputs = model.generate(
        inputs,
        max_length=512,
        num_return_sequences=1,
        num_beams=1,
        early_stopping=True
    )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    labels = output_text.split('=>')[-1].strip()

    return labels


result = generate_labels("Severe pain abdomen 4 months ago . Abdomen problem . Suddenly it happen . Severe problem . Single attack . chronic calculus cholecystitis with chronic appendicitis . Treatment type OPERATIVE . Surgery type CHOLECYSTECTOMY + APPENDICECTOMY . Procedure follow CHOLECYSTECTOMY + APPENDICECTOMY . Anesthic type General . Anes smooth recovery . GB contracted multiple cal. CBD normal Appendix chronically inflammed . Follow-up date 29 April 2000 . ")

def parse_output(output):
    lines = output.split(' ')
    labeled_sentences = {}
    current_label = None
    for line in lines:
        if ':' in line:
            current_label, sentence = line.split(':', 1)
            labeled_sentences[current_label.strip()] = sentence.strip()
        else:
            if current_label:
                labeled_sentences[current_label] += ' ' + line.strip()
    return labeled_sentences

# Example output parsing
parsed_result = parse_output(result)

print(parsed_result)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'ChiefComplaint': ' Ho intermittent pain abdomen 4 months ago', 'PresentProblemLocation': ' Abdomen problem', 'PreProbDuration': ' Complains from 4 months ago', 'PreProbOnset': ' Gradually it happen', 'PreProbSeverity': ' Moderate problem', 'AttackType': ' No attack', 'SmokingHabitType': ' Patient does not have smoking habit', 'PhysicalActivity': ' Light Worker AL', 'ALCHOLIC': ' Does not drink alcohol', 'DietryHabit': ' Non Vegetarian', 'MealFrequency': ' Prefer meals 2 3 times a day', 'VegCookingMedia': ' Prefer oil mustrad oil', 'TreatmentType': ' Treatment type APPENDECTOMY', 'SurgeryType': ' Surgery type APPENDECTOMY', 'SurgeryDurationH': ' nan', 'SurgeryDurationM': ' nan', 'AnesthicTypeName': ' Anesthic type General', 'AnesRecoverySmoothType': ' Anes smooth recovery', 'RecorveryType': ' NORMAL recovery', 'Followupdate': ' Follow-up date 27 April 2014', 'DoctorRemark': ' CHECK UP WITH BIOPSY REPORT AFTER 7 DAYS\nHo intermittent pain abdomen 4 months ago'}


In [5]:
import json

json_data = json.dumps(parsed_result, indent=4)
print(json_data)

{
    "ChiefComplaint": " Ho intermittent pain in epigastric region radiating to back since 6 months",
    "PresentProblemLocation": " Abdomen problem",
    "PreProbDuration": " Complains from 6 months ago",
    "PreProbOnset": " Gradually it happen",
    "PreProbSeverity": " Moderate problem",
    "AttackType": " No attack",
    "SmokingHabitType": " Patient does not have smoking habit",
    "PhysicalActivity": " Light Worker",
    "ALCHOLIC": " Does not drink alcohol",
    "DietryHabit": " Non Vegetarian",
    "MealFrequency": " Prefer meals 2 3 times a day",
    "VegCookingMedia": " Prefer oil mustrad oil",
    "TreatmentType": " Treatment type CHOLECYSTECTOMY",
    "SurgeryType": " Surgery type CHOLECYSTECTOMY",
    "SurgeryDurationH": " nan",
    "SurgeryDurationM": " nan",
    "AnesthicTypeName": " Anesthic type General",
    "AnesRecoverySmoothType": " Anes smooth recovery",
    "RecorveryType": " NORMAL recovery",
    "Followupdate": " Follow-up date 30 March 2013",
    "Doctor

In [7]:
json_data = json.dumps(parsed_result, indent=4)
print(json_data)

{
    "ChiefComplaint": " Ho pain abdomen since one month",
    "PresentProblemLocation": " Abdomen problem",
    "PreProbDuration": " Complains from 1 month ago",
    "PreProbOnset": " Gradually it happen",
    "PreProbSeverity": " nan",
    "AttackType": " No attack",
    "SmokingHabitType": " Patient does not have smoking habit",
    "PhysicalActivity": " Light Worker",
    "ALCHOLIC": " Does not drink alcohol",
    "DietryHabit": " Non Vegetarian",
    "MealFrequency": " Prefer meals 2 3 times a day VegCook",
    "VegCookingMedia": " Prefer oil mustrad oil",
    "TreatmentType": " Treatment type CHOLECYSTECTOMY",
    "SurgeryType": " Surgery type CHOLECYSTECTOMY",
    "SurgeryDurationH": " nan",
    "SurgeryDurationM": " nan",
    "AnesthicTypeName": " Anesthic type General",
    "AnesRecoverySmoothType": " Anes smooth recovery",
    "RecorveryType": " NORMAL recovery",
    "Followupdate": " Follow-up date 31 August 2011",
    "DoctorRemark": " CHECK UP WITH BIOPSY REPORT AFTER 7 D

In [9]:
json_data = json.dumps(parsed_result, indent=4)
print(json_data)

"""Severe pain abdomen 4 months ago . Abdomen problem . Suddenly it happen . Severe problem . Single attack .
chronic calculus cholecystitis with chronic appendicitis . Treatment type OPERATIVE .
Surgery type CHOLECYSTECTOMY + APPENDICECTOMY . Procedure follow CHOLECYSTECTOMY + APPENDICECTOMY .
Anesthic type General . Anes smooth recovery . GB contracted multiple cal. CBD normal Appendix chronically inflammed . Follow-up date 29 April 2000 ."""

{
    "ChiefComplaint": " Ho intermittent pain abdomen 4 months ago",
    "PresentProblemLocation": " Abdomen problem",
    "PreProbDuration": " Complains from 4 months ago",
    "PreProbOnset": " Gradually it happen",
    "PreProbSeverity": " Moderate problem",
    "AttackType": " No attack",
    "SmokingHabitType": " Patient does not have smoking habit",
    "PhysicalActivity": " Light Worker AL",
    "ALCHOLIC": " Does not drink alcohol",
    "DietryHabit": " Non Vegetarian",
    "MealFrequency": " Prefer meals 2 3 times a day",
    "VegCookingMedia": " Prefer oil mustrad oil",
    "TreatmentType": " Treatment type APPENDECTOMY",
    "SurgeryType": " Surgery type APPENDECTOMY",
    "SurgeryDurationH": " nan",
    "SurgeryDurationM": " nan",
    "AnesthicTypeName": " Anesthic type General",
    "AnesRecoverySmoothType": " Anes smooth recovery",
    "RecorveryType": " NORMAL recovery",
    "Followupdate": " Follow-up date 27 April 2014",
    "DoctorRemark": " CHECK UP WITH BIOPSY REPO